In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### AerBackend (QasmSimulator)

In [5]:
from qiskit.providers.aer import AerSimulator

In [6]:
backend = AerSimulator()

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.71679688+0.j), variance=array(11.19474411+0.j), confidence_interval=[((1.5574310667008238+0j), (1.8761626832991762+0j))])

In [7]:
# no parameter case

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

ExpectationValueResult(value=array(1.83203125+0.j), variance=array(11.15776062+0.j), confidence_interval=[((1.6730020581390936+0j), (1.9910604418609064+0j))])

### Exact simulation by SaveExpectationValueVariance

In [8]:
from qiskit.evaluators import ExactExpectationValue

In [9]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
%time expval.evaluate([0, 1, 1, 2, 3, 5])

CPU times: user 6.89 ms, sys: 971 µs, total: 7.86 ms
Wall time: 7.49 ms


ExpectationValueResult(value=array(1.84209213), variance=array(6.43276352), confidence_interval=None)

In [10]:
%time expval.evaluate([1, 1, 2, 3, 5, 8])

CPU times: user 6.13 ms, sys: 2.35 ms, total: 8.48 ms
Wall time: 7.37 ms


ExpectationValueResult(value=array(0.15611897), variance=array(5.35681431), confidence_interval=None)

### Transpile options and Run options

In [11]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.844766+0.j), variance=array(11.14023179+0.j), confidence_interval=[((1.8396758251431204+0j), (1.849856174856879+0j))])

In [12]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=array(1.8203125+0.j), variance=array(11.14194298+0.j), confidence_interval=[((1.764083035748605+0j), (1.876541964251395+0j))])

In [13]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=array(2.16+0.j), variance=array(10.79111111+0.j), confidence_interval=[((1.8698430390175642+0j), (2.450156960982435+0j))])

### Mitigator

Readout mitigator classes [#6485](https://github.com/Qiskit/qiskit-terra/pull/6485) is required


In [ ]:
amat = ...

mitigator = CompleteReadoutMitigator(amat)
PauliExpectationValue(ansatz, observable, backend=backend, mitigator=mitigator)
expval.evaluate([0, 1, 1, 2, 3, 5])